In [1]:
import bs4
import os
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader,TextLoader,JSONLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
from dotenv import load_dotenv
import os

In [9]:
load_dotenv()
api_key = os.environ['OPENAI_API_KEY']

In [12]:
def load_processed_files(directory: str):
    documents = []
    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            file_path = os.path.join(directory, filename)
            
            # Define a custom jq schema to extract both content and metadata
            jq_schema = '.chunks[] | {content: .content, metadata: .metadata}'
            
            loader = JSONLoader(
    file_path=file_path,
    jq_schema=jq_schema,
    content_key="content",
    metadata_func=lambda metadata, _: {**metadata, "source": file_path}  # Add the second argument
)
            
            documents.extend(loader.load())
    
    return documents
loaded_documents = load_processed_files('../pages/json')

In [13]:
from langchain.schema import Document 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# Container for split documents
split_documents = []

# Loop through each loaded document
for doc in loaded_documents:
    # Extract content and metadata
    content = doc.page_content
    metadata = doc.metadata

    
    # Split the content
    split_content = text_splitter.split_text(content)
    
    # Re-associate each split with the original metadata
    for split in split_content:
        split_documents.append({
            'page_content': split,
            'metadata': metadata  # Keep the same metadata for each chunk
        })
def flatten_metadata(metadata):
    # Flatten metadata by joining key-value pairs into a single string
    return {key: str(value) if isinstance(value, dict) else value for key, value in metadata.items()}

# Convert split_documents (which are dictionaries) into Document objects with flattened metadata
documents_as_objects = [
    Document(
        page_content=doc['page_content'], 
        metadata=flatten_metadata(doc['metadata'])  # Flatten the metadata
    )
    for doc in split_documents
]

In [14]:
vectorstore = Chroma.from_documents(documents=documents_as_objects, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

In [15]:
prompt = hub.pull("rlm/rag-prompt")

c:\Users\sudhe\Desktop\Projects\Chatbot\langchain_env\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [16]:
prompt.format_prompt(```You are a helpful assistant who provides detailed and accurate information.
Context: {context}
Question: {question}
Answer:``)

SyntaxError: invalid syntax (3610404506.py, line 1)

In [17]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [18]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Question
response = rag_chain.invoke("Where is USF located?")

In [19]:
response

'USF is located in the vibrant and diverse Tampa Bay region, with campuses in Tampa, St. Petersburg, and Sarasota-Manatee. The campuses comprise more than 1,600 acres and nearly 12 million square feet of building space, with areas boasting coastal, bayfront, and riverfront views.'